In [ ]:
using Matching

In [ ]:
urls = [
    "https://raw.githubusercontent.com/EikiTakigawa/Deferred-Acceptance/a74feec27a0720a4821cfd3827701201d73072b9/DA_Many_to_1.jl",
    "https://raw.githubusercontent.com/IoriS/Ex03/b2623c18aa4c24571cb2028339283450721c8ff2/deferred_acceptance.jl",
    "https://raw.githubusercontent.com/keiikegami/DA_Julia/435e4fb243faf371dc9d3db2c4006e4b0ff64730/ikegamida_mm.jl",
    "https://raw.githubusercontent.com/M-okb/DA-Matching/793c2eceb3a4b9b88c074a98da715a344bb9cf13/many_to_one.jl",
    "https://raw.githubusercontent.com/myuuuuun/oyama_seminar2016/31cc89df572213ac1d7dc4da320b8d2697932fb1/exercise/ex03/matching.jl",
    "https://raw.githubusercontent.com/NlGG/Economics/c9ac87dc1ce2acdbf08a30be89986e2124685e86/deferred_acceptance.jl",
    "https://raw.githubusercontent.com/nswa17/DA_alg.jl/522791c03866a812a610de68618ff105b33bd55c/da.jl",
    "https://raw.githubusercontent.com/oyataku1/Matching/4298b98e530e326aba05fbae6c9fdba39a058a8a/deferred_acceptance.jl",
    "https://raw.githubusercontent.com/SUZUKITAISHI/matching/701096f5bc327a4f619e4f9985cc4674afb6869c/my_Gale_Shap.jl",
    "https://raw.githubusercontent.com/13tsuyoshi/matching/79049ddc672d9795bec6bceab8209cf40a974d30/Many_to_one.jl",
]
function_names = [
    "deferred_acceptance",
    "deferred_acceptance",
    "ikegamida_mm",
    "okb_DA_algo",
    "Matching.gale_shapley_T",
    "deferred_acceptance",
    "DA.call_match",
    "deferred_acceptance",
    "my_Gale_Shap",
    "deferred_acceptance",
];

In [ ]:
file_names = [split(url, "/")[end] for url in urls]
dir_names = [split(url, "/")[4] for url in urls]
paths = [join((dir_name, file_name), "/")
    for (dir_name, file_name) in zip(dir_names, file_names)];

In [ ]:
module_names = [
    replace(
        replace(dir_name, "-", "_"),
        r"^[1-9]+", ""
    )
    for dir_name in dir_names
];

In [ ]:
function download_files(overwrite::Bool=false)
    for (url, dir_name, path) in zip(urls, dir_names, paths)
        if !isdir(dir_name)
            mkdir(dir_name)
        end
        
        if !isfile(path) || overwrite
            download(url, path)
        end
    end
end

In [ ]:
download_files(true)

In [ ]:
function load_module(module_name, path)
    module_name = parse(module_name)
    eval(Expr(:toplevel, :(
        module ($module_name)
        include($path)
        end
    ), module_name, path))
end

In [ ]:
for (module_name, path) in zip(module_names, paths)
    load_module(module_name, path)
end

In [ ]:
module oyamad
deferred_acceptance = Main.Matching.deferred_acceptance
end

In [ ]:
functions = Dict{ASCIIString, Function}()

for (module_name, function_name) in zip(module_names, function_names)
    eval(parse(
        "functions[\"" * module_name * "\"] = " * module_name * "." * function_name
    ))
end

functions["oyamad"] = oyamad.deferred_acceptance;

In [ ]:
functions

## Testing One-to-One

In [ ]:
m, n = 10, 5
srand(1234)
m_prefs, f_prefs = random_prefs(m, n)

prop_matches_oy, resp_matches_oy = oyamad.deferred_acceptance(m_prefs, f_prefs)

for (k, fn) in functions
    println(k)
    try
        @time fn(m_prefs, f_prefs)
    catch
        println("  NA")
        continue
    end
    @time prop_matches, resp_matches = fn(m_prefs, f_prefs)
    @time fn(m_prefs, f_prefs)
    if prop_matches == prop_matches_oy && resp_matches == resp_matches_oy
        println("  OK")
    else
        println("  returned: $prop_matches; expected: $prop_matches_oy")
        println("  returned: $resp_matches; expected: $resp_matches_oy")
    end
end

## Testing Many-to-One

In [ ]:
m, n = 10, 5
c = 2  # Number of caps
srand(1234)
m_prefs, f_prefs = random_prefs(m, n)
caps = Array(Int, n)
fill!(caps, c)

prop_matches_oy, resp_matches_oy, indptr_oy =
    oyamad.deferred_acceptance(m_prefs, f_prefs, caps)
for j in 1:n
    sort!(sub(resp_matches_oy,
              indptr_oy[j]:indptr_oy[j+1]-1)
    )
end

for (k, fn) in functions
    println(k)
    try
        @time fn(m_prefs, f_prefs, caps)
    catch
        println("  NA")
        continue
    end
    @time prop_matches, resp_matches, indptr = fn(m_prefs, f_prefs, caps)
    @time fn(m_prefs, f_prefs, caps)
    
    for j in 1:n
        sort!(sub(resp_matches,
                  indptr[j]:indptr[j+1]-1)
        )
    end
    if prop_matches == prop_matches_oy && resp_matches == resp_matches_oy &&
        indptr == indptr_oy
        println("  OK")
    else
        println("  returned: $prop_matches; expected: $prop_matches_oy")
        println("  returned: $resp_matches; expected: $resp_matches_oy")
    end
end

## Performance Comparison

In [ ]:
functions

In [ ]:
function performance(m::Int, n::Int, caps::Vector{Int}, rng::AbstractRNG)
    m_prefs, f_prefs = random_prefs(rng, m, n)
    
    times = Array(Float64, length(functions))
    names = Array(ASCIIString, length(functions))
    #allocs = Array(Int, length(functions))
    
    for (i, (k, fn)) in enumerate(functions)
        time = 0.
        try
            fn(m_prefs, f_prefs, caps)
            fn(m_prefs, f_prefs, caps)
            _, time, _, _ = @timed fn(m_prefs, f_prefs, caps)
        catch
            time = 9999.
        end
        times[i] = time
        names[i] = k
        #allocs[i] = alloc
    end
    
    indices = sortperm(times)
    for i in indices
        println(names[i])
        @printf("  %0.9f seconds\n", times[i])
    end
end

performance(m::Int, n::Int, caps::Vector{Int}, seed::Int) =
    performance(m, n, caps, MersenneTwister(seed))
performance(m::Int, n::Int, caps::Vector{Int}) =
    performance(m, n, caps, Base.GLOBAL_RNG)

In [ ]:
seed = 1234

In [ ]:
m, n = 500, 500
caps = ones(Int, n)  # one-to-one
performance(m, n, caps, seed)

In [ ]:
m = 100
c = 10
n = div(m, c)
caps = Array(Int, n)
fill!(caps, c)
performance(m, n, caps, seed)

In [ ]:
m = 500
c = 50
n = div(m, c)
caps = Array(Int, n)
fill!(caps, c)
performance(m, n, caps, seed)

In [ ]:
m = 1000
c = 50
n = div(m, c)
caps = Array(Int, n)
fill!(caps, c)
performance(m, n, caps, seed)